Attorney General Webscrape Module for group data. <br>
[x] Get all current AG by state and political affiliation in a dataframe. <br>
[x] Get all former AG by state <br>
[x] look up political affiliation by wiki search on name. 

In [2]:
#if you don't haver the wikipedia library, run this first:
#!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11686 sha256=ed8ba898aa0f861182b351e9f9ffcb0adcf9a581db67df1eb22d8a53b37d1224
  Stored in directory: c:\users\yumst\appdata\local\pip\cache\wheels\07\93\05\72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia


In [3]:
#imports
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import wikipedia as wk

# State Attorney Generals
## Current Attorney Generals by state and party

### Method:
Here we used a basic web scrape with _BeautifulSoup_ library. The table from the [wikipedia page]('https://en.wikipedia.org/wiki/State_attorney_general') was parsed and extracted into a dataframe by table element. The resulting table was cleaned and edited so we could join to a larger table to feature in our main dataset.

In [4]:
#URL and Response object instance
url = 'https://en.wikipedia.org/wiki/State_attorney_general' 
res = requests.get(url)
type(res)

requests.models.Response

In [5]:
res.status_code

200

In [6]:
#soup object on response data
soup = BeautifulSoup(res.content, 'html.parser')
table_class= 'wikitable sortable jquery-tablesorter'
table = soup.find('table', attrs={'class':'wikitable'})
trs  = table.find_all('tr')
type(trs)

bs4.element.ResultSet

In [7]:
inp = []

for tr in trs:
    item = []
    ths = tr.find_all('th')
    tds = tr.find_all('td')
    for th in ths:
        a = th.text
        a = str(a).replace("\n", "")
        item.append(a)
    for td in tds:
        a = td.text
        a = str(a).replace("\n", "")
        #a = a.lower()
        item.append(a)
    inp.append(item)

In [8]:
sag = pd.DataFrame(inp[1:], columns = inp[0])

In [9]:
sag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Officeholder    56 non-null     object
 1   State           56 non-null     object
 2   Party           56 non-null     object
 3   Assumed office  56 non-null     object
 4   Term expires    56 non-null     object
 5   Law school      56 non-null     object
 6   Term limits     56 non-null     object
dtypes: object(7)
memory usage: 3.2+ KB


There were too many instances of Attorneys General in this set due to the inclusion of Am. Samoa, Guam, Puerto Rico, U.S.V.I, and Northern Mariana Islands. These were removed due to the absence of their representation in our main dataset FBI crime. 

In [10]:
#drop non_state rows from set. Should only have 51 rows. 
sag = sag[sag['State'] != 'American Samoa']
sag = sag[sag['State'] != 'Guam']
sag = sag[sag['State'] != 'Puerto Rico[7]']
sag = sag[sag['State'] != 'U.S. Virgin Islands']
sag = sag[sag['State'] != 'Northern Mariana Islands']
sag = sag.drop(columns = ['Law school', 'Term limits'])

In [11]:
sag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 55
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Officeholder    51 non-null     object
 1   State           51 non-null     object
 2   Party           51 non-null     object
 3   Assumed office  51 non-null     object
 4   Term expires    51 non-null     object
dtypes: object(5)
memory usage: 2.4+ KB


In [12]:
# rename columns to match that of former AG data.
# Name, State, Party, start of term, end of term
sag = sag.rename(columns={'Officeholder': 'Name', 'State' : 'State', 'Party' : 'party', 'Assumed office' : 'start of term', 'Term expires' : 'end of term'}).copy()
sag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 55
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           51 non-null     object
 1   State          51 non-null     object
 2   party          51 non-null     object
 3   start of term  51 non-null     object
 4   end of term    51 non-null     object
dtypes: object(5)
memory usage: 2.4+ KB


In [13]:
#strip and adjust start of term column making the year an int. 
sag['start of term'] = sag['start of term'].apply(lambda x: int(x[-4:])).copy()

Since the `end of term` series has _'Appointed'_ in the text we need to remove this to allow our conversion method as before. 

In [14]:
sag[sag['end of term'] =='Appointed']

,Name,State,party,start of term,end of term
1,Treg Taylor,Alaska,Republican,2021,Appointed
13,Clare Connors,Hawaii,Democratic,2019,Appointed
32,Andrew BruckActing,New Jersey,Democratic,2021,Appointed
55,Bridget Hill,Wyoming,Republican,2019,Appointed


In [15]:
#removesd AG's that started after the limit of our data(2021).
sag = sag.drop(sag.iloc[1].name).reset_index()


In [16]:
sag[sag['end of term'] =='Appointed']

,index,Name,State,party,start of term,end of term
10,13,Clare Connors,Hawaii,Democratic,2019,Appointed
29,32,Andrew BruckActing,New Jersey,Democratic,2021,Appointed
49,55,Bridget Hill,Wyoming,Republican,2019,Appointed


In [17]:
sag = sag.drop(sag.iloc[29].name)

In [18]:
sag[sag['end of term'] =='Appointed']

,index,Name,State,party,start of term,end of term
10,13,Clare Connors,Hawaii,Democratic,2019,Appointed
49,55,Bridget Hill,Wyoming,Republican,2019,Appointed


In [19]:
sag[sag['start of term'] >= 2021]

,index,Name,State,party,start of term,end of term
3,5,Rob Bonta,California,Democratic,2021,2023
13,16,Todd Rokita,Indiana,Republican,2021,2025
25,28,Austin Knudsen,Montana,Republican,2021,2025
28,31,John Formella,New Hampshire,Republican,2021,2025 (Appointed)
35,39,John O'Connor,Oklahoma,Republican,2021,2023


In [20]:
sag = sag[sag['start of term'] < 2021].copy().reset_index()

In [21]:
sag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   level_0        44 non-null     int64 
 1   index          44 non-null     int64 
 2   Name           44 non-null     object
 3   State          44 non-null     object
 4   party          44 non-null     object
 5   start of term  44 non-null     int64 
 6   end of term    44 non-null     object
dtypes: int64(3), object(4)
memory usage: 2.5+ KB


In [22]:
#replaces 'appointed target word with string for subsequent conversion'
sag = sag.replace({'Appointed': '2020'})

In [23]:
#check for 'Appointed' word 
sag[sag['end of term'] =='Appointed']

,level_0,index,Name,State,party,start of term,end of term


In [24]:
#strip and convert [end of term] to int representing the year
sag['end of term'] = sag['end of term'].apply(lambda x: int(x[:4])).copy()

In [25]:
sag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   level_0        44 non-null     int64 
 1   index          44 non-null     int64 
 2   Name           44 non-null     object
 3   State          44 non-null     object
 4   party          44 non-null     object
 5   start of term  44 non-null     int64 
 6   end of term    44 non-null     int64 
dtypes: int64(4), object(3)
memory usage: 2.5+ KB


In [26]:
sag['party'].value_counts()

Democratic    23
Republican    21
Name: party, dtype: int64

In [27]:
sag[sag['State'] == 'California']

,level_0,index,Name,State,party,start of term,end of term


In [71]:
sag['party'] = sag['party'].map(lambda x: x.replace('Democratic','Democrat')) 

---

.

## Former Attorney Generals
---
Below we begin the hunt for historical (aka: former) attorney generals for each state going back as far as we can. 
Our data was scraped from [National Association of Attorney Generals](https://www.naag.org/news-resources/research-data/). We chose this due to a uniformity of data representation in a table that could easily be compiled into a pandas DataFrame. 

In [28]:
def get_ag(state):
    '''
    Function takes a URL to go to and scrape the table of past AG's for a state.
    Meant to be run with loop. 
    '''
    if state == 'Louisiana':
        url_state = 'https://www.naag.org/attorneys-general/past-attorneys-general/louisiana-former-attorney-general/'
    
    elif state == 'Connecticut':
        url_state = 'https://www.naag.org/attorneys-general/past-attorneys-general/connecticutformer-attorneys-general/'
    
    else:    
        url_state = f'https://www.naag.org/attorneys-general/past-attorneys-general/{state}-former-attorneys-general/'
   
    #print(f'Working on {state} to import') #diagnostic line
    #Diagnostic print line
    #print(f'url is {url_state}')
    
    time.sleep(1)
    res2 = requests.get(url_state, headers={'User-Agent': 'Mozilla/5.0'})
    #print('Code', res.status_code) #diagnostic line of code
    
    try:
        soup2 = BeautifulSoup(res2.content, 'html.parser')
        table2 = soup2.find('tbody')
        trs2  = table2.find_all('tr')
        states = []
        for tr in trs2:
            item = [f'{state}']
            tds = tr.find_all('td')
            for td in tds:
                a = td.text
                item.append(a)
            states.append(item)
        states_df = pd.DataFrame(states, columns = ['State', 'Name', 'Term'])
    except:
        print(f'**************** \
        There was an error with {state}, please correct \
        ***************')
        states = []
    return states_df

Since we removed two states having had Attorney General officers begin after our measurement period, the states were dropped. This left us with 49 states currently. 
We will be filling in the gaps for each year below, since this data came from one source:
[Wikipedia](https://en.wikipedia.org/wiki/State_attorney_general)

In [29]:
# list of states from above to help us get a new DtatFrame, Making sure that the two states we dropped are included to bring the total to 51
lst_states = list(sag['State'])
lst_states.append('Alaska')
lst_states.append('New Jersey')
lst_states.append('California')
lst_states.append('Indiana')
lst_states.append('Montana')
lst_states.append('New Hampshire')
lst_states.append('Oklahoma')
print(len(lst_states))
lst_states = sorted(lst_states)


51


In [30]:
main = []
for state in lst_states:
    out = get_ag(state)
    main.append(out)
    print(f'{state} import completed')

Alabama import completed
Alaska import completed
Arizona import completed
Arkansas import completed
California import completed
Colorado import completed
Connecticut import completed
Delaware import completed
District of Columbia import completed
Florida import completed
Georgia import completed
Hawaii import completed
Idaho import completed
Illinois import completed
Indiana import completed
Iowa import completed
Kansas import completed
Kentucky import completed
Louisiana import completed
Maine import completed
Maryland import completed
Massachusetts import completed
Michigan import completed
Minnesota import completed
Mississippi import completed
Missouri import completed
Montana import completed
Nebraska import completed
Nevada import completed
New Hampshire import completed
New Jersey import completed
New Mexico import completed
New York import completed
North Carolina import completed
North Dakota import completed
Ohio import completed
Oklahoma import completed
Oregon import comple

Having created a big list of appended data representing a state's historical record of Attorneys General, we create a DataFrame to continue preprocessing and cleaning. 

### Cleaning DataFrame of all historic Attorneys General


In [31]:
former_ag = pd.concat(main, ignore_index = True)

In [32]:
former_ag[['start of term', 'end of term']] = former_ag['Term'].str.split('–', expand = True)
former_ag[['start of term', 'end of term']] = former_ag['Term'].str.split('– ', expand = True)

In [33]:
former_ag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2068 entries, 0 to 2067
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          2068 non-null   object
 1   Name           2068 non-null   object
 2   Term           2068 non-null   object
 3   start of term  2068 non-null   object
 4   end of term    1938 non-null   object
dtypes: object(5)
memory usage: 80.9+ KB


Need to edit output. Some states appoint or otherwise seat an Attorney Gen. Turn all strings to numbers. I will do this in a function with a try/except block to handle odd strings here and there. There are a number of people have no end data, will need to look at that too. 

In [34]:
def to_num(string):
    '''
    Accepts a string and turns it into a number if possible. If not possible the location of unconverted numbers
    returned in a list with location
    '''
    try:
        alpha = int(string)
        
    except:
        alpha = print(f'the string {string} could not be converted')
    
    return alpha

In [35]:
former_ag['start of term'].apply(to_num)

the string 1991 -1999 could not be converted
the string 1892 -1895 could not be converted
the string 1815-1817 could not be converted
the string  could not be converted
the string 1981- 1984 could not be converted
the string October 2006 could not be converted
the string Appointed: 1926 could not be converted
the string Appointed: 2013, Elected: 2014  could not be converted


0       2010.0
1       2004.0
2       1997.0
3       1995.0
4       1991.0
         ...  
2063    1905.0
2064    1898.0
2065    1895.0
2066    1891.0
2067    1886.0
Name: start of term, Length: 2068, dtype: float64

Investigate each of the strange ones: correcting the ones that are wrong, based on Wikipedia, and dropping the ones older than 1979

In [36]:
former_ag[former_ag['start of term'] == '1991 -1999']

,State,Name,Term,start of term,end of term
130,Arkansas,Winston Bryant,1991 -1999,1991 -1999,None


In [37]:
former_ag.iloc[130]['start of term'] = 1991
former_ag.iloc[130]['end of term'] = 1999

In [38]:
former_ag[former_ag['start of term'] == '1981- 1984']

,State,Name,Term,start of term,end of term
1229,New Hampshire,Stephen E Merrill,1981- 1984,1981- 1984,None


In [39]:
former_ag.iloc[1229]['start of term'] = 1985
former_ag.iloc[1229]['end of term'] = 1989

In [40]:
former_ag[former_ag['start of term'] == 'October 2006']

,State,Name,Term,start of term,end of term
1829,Tennessee,Michael E. Moore,October 2006,October 2006,None


In [41]:
former_ag.iloc[1829]['Name'] = 'Robert E. Cooper, Jr.'
former_ag.iloc[1829]['start of term'] = 2006
former_ag.iloc[1829]['end of term'] = 2014

In [42]:
former_ag[former_ag['start of term'] == 'Appointed: 2013, Elected: 2014 ']

,State,Name,Term,start of term,end of term
1865,Utah,Sean Reyes,"Appointed: 2013, Elected: 2014 – Present","Appointed: 2013, Elected: 2014",Present


In [43]:
sag[sag['State'] == 'Utah']

,level_0,index,Name,State,party,start of term,end of term
37,43,49,Sean Reyes,Utah,Republican,2013,2025


In [44]:
#reyes is a current AG
former_ag = former_ag.drop(1865)

In [45]:
former_ag[former_ag['start of term'] == '1892 -1895']

,State,Name,Term,start of term,end of term
306,Delaware,John R. Nicholson,1892 -1895,1892 -1895,None


In [46]:
former_ag = former_ag.drop(306)

In [47]:
former_ag[former_ag['start of term'] == '1815-1817']

,State,Name,Term,start of term,end of term
808,Louisiana,Etienne Mazareu,1815-1817,1815-1817,None


In [48]:
former_ag = former_ag.drop(808)

In [49]:
former_ag[former_ag['start of term'] == 'Appointed: 1926']

,State,Name,Term,start of term,end of term
1843,Tennessee,Charles L. Cornelius,Appointed: 1926,Appointed: 1926,None


In [50]:
former_ag = former_ag.drop(1843)

In [51]:
former_ag[former_ag['start of term'] == '']

,State,Name,Term,start of term,end of term
946,Massachusetts,Office Abolished 1843 – 1849,,,None


In [52]:
former_ag = former_ag.drop(946)

In [53]:
former_ag['start of term'] = former_ag['start of term'].apply(to_num)

In [54]:
former_ag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 0 to 2067
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          2063 non-null   object
 1   Name           2063 non-null   object
 2   Term           2063 non-null   object
 3   start of term  2063 non-null   int64 
 4   end of term    1940 non-null   object
dtypes: int64(1), object(4)
memory usage: 96.7+ KB


---

Now to adjust the end of term series

In [55]:
former_ag['end of term'].apply(to_num)

the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not

0       2017.0
1       2010.0
2       2004.0
3       1997.0
4       1995.0
         ...  
2063    1911.0
2064    1905.0
2065    1898.0
2066    1895.0
2067    1891.0
Name: end of term, Length: 2063, dtype: float64

Our function has provided some useful information It appears as though we have three values; Numbers, _None_ and _`'Present'`_ First, we will deal with 'Present' by checking our `sag` DataFrame 
The following states `end of term` will be adjusted to `2020` since new AG officers began in these states in 2021; _Alsaska, New Jersey, California, Indiana, Montana, New Hampshire, Oklahoma._


In [56]:
former_ag[former_ag['State'] == 'Oklahoma']

,State,Name,Term,start of term,end of term
1538,Oklahoma,Mike Hunter,2017 – Present,2017,Present
1539,Oklahoma,Cara Rodrigues,2017,2017,None
1540,Oklahoma,Scott Pruitt,2011 – 2017,2011,2017
1541,Oklahoma,Drew Edmondson,1995 – 2011,1995,2011
1542,Oklahoma,Susan B. Loving,1991 – 1995,1991,1995
1543,Oklahoma,Robert H. Henry,1987 – 1991,1987,1991
1544,Oklahoma,Mike Turpen,1983 – 1987,1983,1987
1545,Oklahoma,Jan Eric Cartwright,1979 – 1983,1979,1983
1546,Oklahoma,Larry Derryberry,1971 – 1979,1971,1979
1547,Oklahoma,G.T. Blankenship,1967 – 1971,1967,1971


In [57]:
cal = {'State': 'California', 'Name' : 'Xavier Becerra', 'Term': '2017 - 2020', 'start of term' : '2017', 'end of term': '2020'}
former_ag = former_ag.append(cal, ignore_index = True)

In [58]:
former_ag[former_ag['State'] == 'Oklahoma'] = former_ag[former_ag['State'] == 'Oklahoma'].replace({'Present': '2020', None : '2017'})

In [59]:
former_ag[former_ag['end of term'] == 'Present'] 

,State,Name,Term,start of term,end of term
524,Idaho,Lawrence Wasden,2003 – Present,2003,Present
964,Michigan,Dana Nessel,2019 – Present,2019,Present
1248,New Jersey,Gurbir S. Grewal,2018 – Present,2018,Present
1311,New Mexico,Hector Balderas,2015 – Present,2015,Present
1342,New York,Letitia James,2019 – Present,2019,Present
1484,Ohio,Dave Yost,2019 – Present,2019,Present
1553,Oregon,Ellen Rosenblum,2012 – Present,2012,Present
1570,Pennsylvania,Josh Shapiro,2017 – Present,2017,Present
1659,Rhode Island,Peter F. Neronha,2019 – Present,2019,Present
1747,South Carolina,Alan Wilson,2011 – Present,2011,Present


In [60]:
former_ag[former_ag['end of term'] == 'Present']  = former_ag[former_ag['end of term'] == 'Present'].replace({'Present': '2020'})

We will use the drop duplicates process in a moment to handle anyone who is duplicated across the frames. 

In [61]:
former_ag['end of term'].apply(to_num)

the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not be converted
the string None could not

0       2017.0
1       2010.0
2       2004.0
3       1997.0
4       1995.0
         ...  
2059    1905.0
2060    1898.0
2061    1895.0
2062    1891.0
2063    2020.0
Name: end of term, Length: 2064, dtype: float64

In [62]:
former_ag = former_ag.dropna()

In [63]:
former_ag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1942 entries, 0 to 2063
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          1942 non-null   object
 1   Name           1942 non-null   object
 2   Term           1942 non-null   object
 3   start of term  1942 non-null   object
 4   end of term    1942 non-null   object
dtypes: object(5)
memory usage: 91.0+ KB


In [64]:
former_ag['start of term'] = former_ag['start of term'].apply(to_num)
former_ag['end of term'] = former_ag['end of term'].apply(to_num)

In [65]:
former_ag.head()

,State,Name,Term,start of term,end of term
0,Alabama,Luther Strange,2010 – 2017,2010,2017
1,Alabama,Troy King,2004 – 2010,2004,2010
2,Alabama,Bill Pryor,1997 – 2004,1997,2004
3,Alabama,Jeff Sessions,1995 – 1997,1995,1997
4,Alabama,Jimmy Evans,1991 – 1995,1991,1995


In [66]:
#Check to ensure that the string for year has converted to int. 
former_ag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1942 entries, 0 to 2063
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          1942 non-null   object
 1   Name           1942 non-null   object
 2   Term           1942 non-null   object
 3   start of term  1942 non-null   int64 
 4   end of term    1942 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 91.0+ KB


In [67]:
#setup wiki utility to call for each row. 
def get_party(name):
    '''
    Accepts a dataframe row as argument. Intended to be called in a .apply() manner to search over wikipedia data to obtain values for df['party'] column.
    '''
    
    #search wikipedia for named person

    #ag = wk.page(f'{name} attorney general', redirect = True)

    try:
        ag = wk.page(f'{name} attorney general', redirect = True)
        for cat in ag.categories:
            if f'Republicans' in cat:
                party = 'Republican'
                return party
            elif 'Democrats' in cat:
                party = 'Democrat'
                return party
    except:
        party = np.nan
        return party


In [73]:
ag_since_1960 = former_ag[former_ag['start of term']>1960].copy()

In [74]:
ag_since_1960.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 2063
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          496 non-null    object
 1   Name           496 non-null    object
 2   Term           496 non-null    object
 3   start of term  496 non-null    int64 
 4   end of term    496 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 23.2+ KB


In [75]:
ag_since_1960['party'] = np.nan

In [76]:
ag_since_1960.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 2063
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   State          496 non-null    object 
 1   Name           496 non-null    object 
 2   Term           496 non-null    object 
 3   start of term  496 non-null    int64  
 4   end of term    496 non-null    int64  
 5   party          0 non-null      float64
dtypes: float64(1), int64(2), object(3)
memory usage: 27.1+ KB


In [77]:
ag_since_1960['party'] = ag_since_1960['Name'].apply(get_party)

C:\Users\yumst\anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\yumst\anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [78]:
ag_since_1960.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 2063
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          496 non-null    object
 1   Name           496 non-null    object
 2   Term           496 non-null    object
 3   start of term  496 non-null    int64 
 4   end of term    496 non-null    int64 
 5   party          294 non-null    object
dtypes: int64(2), object(4)
memory usage: 27.1+ KB


In [79]:
299/498


0.6004016064257028

In [80]:
ag_since_1960['party'].value_counts()

Democrat      161
Republican    133
Name: party, dtype: int64

In [81]:
ag_since_1960[ag_since_1960['party'].isnull()]['State'].value_counts()

District of Columbia    19
Alaska                  17
Pennsylvania            11
Hawaii                  10
New Jersey              10
Tennessee                9
New Hampshire            9
Wyoming                  7
Maine                    6
Nevada                   6
Kentucky                 5
Indiana                  5
Utah                     5
Oregon                   4
Idaho                    4
Ohio                     4
Arkansas                 4
Nebraska                 4
Montana                  4
Arizona                  4
South Dakota             3
West Virginia            3
Rhode Island             3
Illinois                 3
Virginia                 3
New Mexico               3
California               3
Iowa                     2
Massachusetts            2
Missouri                 2
Wisconsin                2
South Carolina           2
Kansas                   2
Texas                    2
Colorado                 2
Oklahoma                 2
Florida                  2
N

In [82]:
ag_since_1960[ag_since_1960['party'].isnull()]['State'].value_counts()

District of Columbia    19
Alaska                  17
Pennsylvania            11
Hawaii                  10
New Jersey              10
Tennessee                9
New Hampshire            9
Wyoming                  7
Maine                    6
Nevada                   6
Kentucky                 5
Indiana                  5
Utah                     5
Oregon                   4
Idaho                    4
Ohio                     4
Arkansas                 4
Nebraska                 4
Montana                  4
Arizona                  4
South Dakota             3
West Virginia            3
Rhode Island             3
Illinois                 3
Virginia                 3
New Mexico               3
California               3
Iowa                     2
Massachusetts            2
Missouri                 2
Wisconsin                2
South Carolina           2
Kansas                   2
Texas                    2
Colorado                 2
Oklahoma                 2
Florida                  2
N

In [83]:
ag_since_1960[ag_since_1960['State']== 'Massachusetts']

,State,Name,Term,start of term,end of term,party
906,Massachusetts,Martha Coakley,2007 – 2015,2007,2015,Democrat
907,Massachusetts,Thomas Reilly,1999 – 2007,1999,2007,Democrat
908,Massachusetts,L. Scott Harshbarger,1991 – 1999,1991,1999,None
909,Massachusetts,James M. Shannon,1987 – 1991,1987,1991,Democrat
910,Massachusetts,Francis X. Bellotti,1975 – 1987,1975,1987,Democrat
911,Massachusetts,Robert H. Quinn,1969 – 1975,1969,1975,NaN
912,Massachusetts,Elliot L. Richardson,1967 – 1969,1967,1969,Republican
914,Massachusetts,Edward W. Brooke,1963 – 1967,1963,1967,Republican


In [84]:
ag_since_1960[ag_since_1960['State']== 'Massachusetts']['party'].mode().iloc[0]

'Democrat'

In [85]:
ag_since_1960[ag_since_1960['State']== 'Massachusetts'] = ag_since_1960[ag_since_1960['State']== 'Massachusetts'].fillna(ag_since_1960[ag_since_1960['State']== 'Massachusetts']['party'].mode().iloc[0])

In [86]:
ag_since_1960[ag_since_1960['State']== 'Massachusetts']

,State,Name,Term,start of term,end of term,party
906,Massachusetts,Martha Coakley,2007 – 2015,2007,2015,Democrat
907,Massachusetts,Thomas Reilly,1999 – 2007,1999,2007,Democrat
908,Massachusetts,L. Scott Harshbarger,1991 – 1999,1991,1999,Democrat
909,Massachusetts,James M. Shannon,1987 – 1991,1987,1991,Democrat
910,Massachusetts,Francis X. Bellotti,1975 – 1987,1975,1987,Democrat
911,Massachusetts,Robert H. Quinn,1969 – 1975,1969,1975,Democrat
912,Massachusetts,Elliot L. Richardson,1967 – 1969,1967,1969,Republican
914,Massachusetts,Edward W. Brooke,1963 – 1967,1963,1967,Republican


In [87]:
ag_since_1960.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 2063
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          496 non-null    object
 1   Name           496 non-null    object
 2   Term           496 non-null    object
 3   start of term  496 non-null    int64 
 4   end of term    496 non-null    int64 
 5   party          296 non-null    object
dtypes: int64(2), object(4)
memory usage: 27.1+ KB


In [88]:
ag_since_1960 = ag_since_1960.fillna('Unknown')

In [89]:
ag_since_1960.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 2063
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          496 non-null    object
 1   Name           496 non-null    object
 2   Term           496 non-null    object
 3   start of term  496 non-null    int64 
 4   end of term    496 non-null    int64 
 5   party          496 non-null    object
dtypes: int64(2), object(4)
memory usage: 27.1+ KB


In [90]:
ag_since_1960

,State,Name,Term,start of term,end of term,party
0,Alabama,Luther Strange,2010 – 2017,2010,2017,Republican
1,Alabama,Troy King,2004 – 2010,2004,2010,Republican
2,Alabama,Bill Pryor,1997 – 2004,1997,2004,Democrat
3,Alabama,Jeff Sessions,1995 – 1997,1995,1997,Republican
4,Alabama,Jimmy Evans,1991 – 1995,1991,1995,Unknown
...,...,...,...,...,...,...
2040,Wyoming,Clarence A. Brimmer,1971 – 1974,1971,1974,Republican
2041,Wyoming,James E. Barrett,1967 – 1971,1967,1971,Unknown
2042,Wyoming,Dean W. Borthwick,1965 – 1967,1965,1967,Republican
2043,Wyoming,John F. Raper,1963 – 1965,1963,1965,Republican


In [91]:
ag_since_1960[ag_since_1960['party'] == 'Unknown']

,State,Name,Term,start of term,end of term,party
4,Alabama,Jimmy Evans,1991 – 1995,1991,1995,Unknown
48,Alaska,Jahna Lindemuth,2016 – 2018,2016,2018,Unknown
49,Alaska,Craig W. Richards,2014 – 2016,2014,2016,Unknown
50,Alaska,Michael C. Geraghty,2012 – 2014,2012,2014,Unknown
51,Alaska,John J Burns,2010 – 2012,2010,2012,Unknown
...,...,...,...,...,...,...
2032,Wyoming,Patrick J. Crank,2003 – 2007,2003,2007,Unknown
2033,Wyoming,Hoke MacMillan,2002 – 2002,2002,2002,Unknown
2035,Wyoming,William U. Hill,1995 – 1998,1995,1998,Unknown
2037,Wyoming,A.G. McClintock,1983 – 1986,1983,1986,Unknown


### Combining Current and former Attorney General dataframes. 

In [92]:
print(sag.shape)
print(ag_since_1960.shape)

(44, 7)
(496, 6)


In [93]:
print(sag.columns)

Index(['level_0', 'index', 'Name', 'State', 'party', 'start of term',
       'end of term'],
      dtype='object')


In [94]:
print(ag_since_1960.columns)

Index(['State', 'Name', 'Term', 'start of term', 'end of term', 'party'], dtype='object')


In [95]:
sag = sag.drop(columns=['level_0', 'index'])

In [96]:
ag_since_1960 = ag_since_1960.drop(columns=['Term'])

In [97]:
print(sag.shape)
print(ag_since_1960.shape)

(44, 5)
(496, 5)


In [98]:
print(sag.columns)
print(ag_since_1960.columns)

Index(['Name', 'State', 'party', 'start of term', 'end of term'], dtype='object')
Index(['State', 'Name', 'start of term', 'end of term', 'party'], dtype='object')


In [99]:
cols = ag_since_1960.columns.tolist()

In [100]:
sag = sag[cols]

In [101]:
print(sag.columns)
print(ag_since_1960.columns)

Index(['State', 'Name', 'start of term', 'end of term', 'party'], dtype='object')
Index(['State', 'Name', 'start of term', 'end of term', 'party'], dtype='object')


In [102]:
sag.head(3)

,State,Name,start of term,end of term,party
0,Alabama,Steve Marshall,2017,2023,Republican
1,Arizona,Mark Brnovich,2015,2023,Republican
2,Arkansas,Leslie Rutledge,2015,2023,Republican


In [103]:
ag_since_1960.head(3)

,State,Name,start of term,end of term,party
0,Alabama,Luther Strange,2010,2017,Republican
1,Alabama,Troy King,2004,2010,Republican
2,Alabama,Bill Pryor,1997,2004,Democrat


In [104]:
frames = [sag, ag_since_1960]

In [105]:
atty_general = pd.concat(frames).reset_index()

In [106]:
atty_general = atty_general.drop(columns = ['index'])

atty_generalcombined DataFrame gets a few transformations and cleaning tasks 
1.check for duplicated entries. 


Checking for duplicated entries

In [108]:
dupes = atty_general[atty_general.duplicated()]

In [109]:
dupes

,State,Name,start of term,end of term,party
325,New Jersey,Stuart Rabner,2006,2007,Democrat
448,Tennessee,"Robert E. Cooper, Jr.",2006,2014,Unknown
524,Wyoming,Bridget Hill,2019,2020,Republican


Only duplicated 3 instances from frame 1 (sag) to frame 2 (ag_since_1960)

In [110]:
atty_general = atty_general.drop_duplicates()

In [111]:
dupes = atty_general[atty_general.duplicated()]
dupes

,State,Name,start of term,end of term,party


In [112]:
atty_general.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 537 entries, 0 to 539
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          537 non-null    object
 1   Name           537 non-null    object
 2   start of term  537 non-null    int64 
 3   end of term    537 non-null    int64 
 4   party          537 non-null    object
dtypes: int64(2), object(3)
memory usage: 25.2+ KB


In [113]:
atty_general[atty_general['end of term'] >= 2020]['Name'].value_counts()

Dave Yost                  2
Josh Shapiro               2
Letitia James              2
Ellen Rosenblum            2
Josh Kaul                  2
Ken Paxton                 2
Alan Wilson                2
Patrick Morrisey           2
Bob Ferguson               2
Dana Nessel                2
Lawrence Wasden            2
Mark Herring               2
Hector Balderas            2
Kathy Jennings             1
Tim Fox                    1
Xavier Becerra             1
Mark Brnovich              1
Kwame Raoul                1
Karl Racine                1
Derek Schmidt              1
Lynn Fitch                 1
Curtis T. Hill             1
Sean Reyes                 1
Herbert H. Slatery, III    1
Leslie Rutledge            1
Steve Marshall             1
Kevin G. Clarkson          1
Peter F. Neronha           1
Josh Stein                 1
Gordon MacDonald           1
Clare Connors              1
Ashley Moody               1
Phil Weiser                1
Peter Neronha              1
Jeff Landry   

In [114]:
atty_general[atty_general['Name'] == 'Josh Kaul']


,State,Name,start of term,end of term,party
42,Wisconsin,Josh Kaul,2019,2023,Democrat
515,Wisconsin,Josh Kaul,2019,2020,Democrat


In [115]:
dupes2 = atty_general[atty_general.duplicated(['State','Name'], keep = 'last')]
to_drop = dupes2.index.tolist()
print(dupes2)

              State               Name  start of term  end of term       party
10            Idaho    Lawrence Wasden           2003         2023  Republican
12             Iowa         Tom Miller           1995         2023    Democrat
19         Michigan        Dana Nessel           2019         2023    Democrat
25       New Mexico    Hector Balderas           2015         2023    Democrat
26         New York      Letitia James           2019         2023    Democrat
29             Ohio          Dave Yost           2019         2023  Republican
30           Oregon    Ellen Rosenblum           2012         2025    Democrat
31     Pennsylvania       Josh Shapiro           2017         2025    Democrat
33   South Carolina        Alan Wilson           2011         2023  Republican
36            Texas         Ken Paxton           2015         2023  Republican
39         Virginia       Mark Herring           2014         2022    Democrat
40       Washington       Bob Ferguson           201

In [116]:
atty_general[atty_general['Name'] == 'Norman C. Gorsuch']

,State,Name,start of term,end of term,party
68,Alaska,Norman C. Gorsuch,1982,1985,Unknown
71,Alaska,Norman C. Gorsuch,1973,1974,Unknown


In [117]:
to_drop

[10, 12, 19, 25, 26, 29, 30, 31, 33, 36, 39, 40, 41, 42, 68, 232]

In [118]:
for index in to_drop:
    atty_general = atty_general.drop(index)

In [119]:
atty_general[atty_general.duplicated(['State','Name'], keep = 'last')]

,State,Name,start of term,end of term,party


In [120]:
atty_general[atty_general['end of term'] > 2020]['end of term'].value_counts()

2023    21
2021     3
2024     3
2025     3
2022     1
Name: end of term, dtype: int64

In [121]:
atty_general[atty_general['end of term'] > 2020] = atty_general[atty_general['end of term'] > 2020].replace({2023: 2020, 2024 : 2020, 2025: 2020, 2022:2020, 2021:2020})

In [122]:
atty_general[atty_general['end of term'] > 2020]['end of term'].value_counts()

Series([], Name: end of term, dtype: int64)

In [123]:
atty_general = atty_general.reset_index()
atty_general = atty_general.drop(columns = ['index'])

In [124]:
atty_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          521 non-null    object
 1   Name           521 non-null    object
 2   start of term  521 non-null    int64 
 3   end of term    521 non-null    int64 
 4   party          521 non-null    object
dtypes: int64(2), object(3)
memory usage: 20.5+ KB


### Expanding the list of Attorney Generals to one row per year per instance. 
In this processing section. Our aim it give each year its own label so it could be used in our predictor series which is also indexed by year since 1979 by state.
1. expand all rows equal to the amount of years a person spent in office. 

1. determine which party had control in a given year. if only one party was identified then that cell has the same value as the political affiliation as the lone member; `Republican`, `Democratic`, or `Unknown`. 
1. if more than one person was in office during a year -and- their political party affiliation are not the same, then the category becomes `Mixed`

In [125]:
output = False
for i in range(len(atty_general)):
    for j in range(atty_general.loc[i,'start of term'], atty_general.loc[i,'end of term']+1):
        #print(clean_ag.loc[i,'Name'],j)
        if output is False:
            output = pd.DataFrame({'State' : atty_general.loc[i, 'State'],
                                   'Name': atty_general.loc[i,'Name'], 
                                   'party': atty_general.loc[i,'party'],
                                   'year': j}, index = [i])
        else:
            output = output.append({'State' : atty_general.loc[i, 'State'],
                                    'Name': atty_general.loc[i,'Name'], 
                                   'party': atty_general.loc[i,'party'],
                                   'year': j}, ignore_index=True)

In [126]:
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3210 entries, 0 to 3209
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   State   3210 non-null   object
 1   Name    3210 non-null   object
 2   party   3210 non-null   object
 3   year    3210 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 100.4+ KB


In [127]:
output.sort_values(by = ['State', 'year']).head(25)

,State,Name,party,year
192,Alabama,Richmond M. Flowers,Democrat,1963
193,Alabama,Richmond M. Flowers,Democrat,1964
194,Alabama,Richmond M. Flowers,Democrat,1965
195,Alabama,Richmond M. Flowers,Democrat,1966
187,Alabama,MacDonald Gallion,Democrat,1967
196,Alabama,Richmond M. Flowers,Democrat,1967
188,Alabama,MacDonald Gallion,Democrat,1968
189,Alabama,MacDonald Gallion,Democrat,1969
190,Alabama,MacDonald Gallion,Democrat,1970
178,Alabama,William J. Baxley,Democrat,1971


Quite a lot of data here. Pruning to match the shape of our basic predictor data. Masking to retrain all rows of 1979 and greater prior to any further transformations. 

In [293]:
ag_since_79 = output[output['year'] > 1978].copy()

In [294]:
ag_since_79.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2394 entries, 0 to 3209
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   State   2394 non-null   object
 1   Name    2394 non-null   object
 2   party   2394 non-null   object
 3   year    2394 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 93.5+ KB


In [295]:
ag_since_79.head()

,State,Name,party,year
0,Alabama,Steve Marshall,Republican,2017
1,Alabama,Steve Marshall,Republican,2018
2,Alabama,Steve Marshall,Republican,2019
3,Alabama,Steve Marshall,Republican,2020
4,Arizona,Mark Brnovich,Republican,2015


In [296]:
states_and_years = ag_since_79.copy()

In [297]:
states_and_years = states_and_years[['State','year']]

In [298]:
states_and_years.drop_duplicates(inplace=True)

In [300]:
states_and_years.sort_values(['State','year'],inplace=True)

In [302]:
states_and_years.reset_index(inplace=True)

In [306]:
states_and_years.drop(columns=['index'], inplace=True)

In [307]:
states_and_years

,State,year
0,Alabama,1979
1,Alabama,1980
2,Alabama,1981
3,Alabama,1982
4,Alabama,1983
...,...,...
2079,Wyoming,2016
2080,Wyoming,2017
2081,Wyoming,2018
2082,Wyoming,2019


In [230]:
def get_ag_control(state, year):
    df = ag_since_79.copy()
    df = df[(df['State']==state) & (df['year']==year)]
    count_R = len(df[df['party']=='Republican'])
    count_D = len(df[df['party']=='Democrat'])
    count_U = len(df[df['party']=='Unknown'])
    
    if count_U > 0:
        return 'Unknown'
    elif count_R > 0 and count_D > 0:
        return 'Mixed'
    elif count_R > 0:
        return 'Republican'
    else:
        return 'Democrat'

In [309]:
control = []
for i in range(len(states_and_years)):
    control.control(get_ag_counts(states_and_years.loc[i,'State'],states_and_years.loc[i,'year']))

In [315]:
states_and_years['control'] = control

In [317]:
states_and_years['control'].value_counts()

Democrat      854
Unknown       663
Republican    519
Mixed          48
Name: control, dtype: int64

In [320]:
states_and_years = pd.get_dummies(states_and_years,columns=['control'])

In [322]:
states_and_years.columns = ['State', 'year', 'ag_Democrat', 'ag_Mixed',
       'ag_Republican', 'ag_Unknown']

In [324]:
states_and_years.shape

(2084, 6)

In [325]:
states_and_years.to_csv('../data/AG.csv',index = False)